In [1]:
import csv 
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import numpy as np
import nltk
import itertools
from nltk.corpus import stopwords
import re
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/kasem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kasem/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [126]:
def pre_process_df_keywords(df):
    for column in df.columns:
        for indx, entry in enumerate(df[column].dropna(axis=0)):
            if len(entry.split()) > 1 and entry[-1] != " ":
                df[column][indx] = entry + " "
def get_two_word(message):
    combined_2_words = []
    for word1, word2 in zip(message,list( message[1:] + list((message[0])))):
        combined_2_words.append(word1 + ' ' + word2)
    return combined_2_words

def pre_process(message):
    stopWords = set(stopwords.words('english'))
    pronouns = ["we", "We","WE","I","me","Me","ME","THEY","they","They","Them","them","THEM","YOU","You","you"]
    result = []
    for word in message.split():
        if word not in stopWords and word not in pronouns:
            result.append(word)
    return (result)
    
def gen_len(iter):
    return sum([1 for _ in iter])


def isMatch_1_word(word1,word2,thresh=80): return (fuzz.QRatio(word1, word2) > thresh)
def isMatch_many_words(word1,message,thresh=90): return (fuzz.partial_ratio(word1, message) >= thresh)

def column_summary(word1, df):
    summary = np.zeros(len(df.columns))
    for col_indx, worker in enumerate(df.columns):
        for word2 in df[worker].dropna(axis=0):
            if isMatch_1_word(word1,word2):
                summary[col_indx]+=1
    return summary


def summarize_one_words(message, first):
    mat = np.zeros((len(message),len(first.columns)))
    for word_indx, word in enumerate(message):    
        mat[word_indx]=(column_summary(word,first)) 
    return mat

def summarize_two_words(message, df):
    mat = np.zeros((len(message),len(df.columns)))
    for col_indx, worker in enumerate(df.columns):
        two_words = df[df[worker].apply(lambda x: len(str(x).split())>1)][worker]
        for word in two_words.dropna(axis=0):
            for word_indx, two_word in enumerate(get_two_word(message)):
                if isMatch_many_words(word, two_word):
                    mat[word_indx][col_indx]+=1
    return mat

def find_workers(raw_message, message, first):
    mat = summarize_one_words(message, first) + summarize_two_words(message, first)
    #print(mat,"\n\n")
    for word_indx, row_word in enumerate(mat[:]):
        if row_word.sum() == 0:
            mat[word_indx] = np.zeros((len(first.columns))); continue
            
        if len(first.columns[row_word != 0]) > 1:
            mat[word_indx] = np.zeros((len(first.columns)))
    
    summary = np.array(mat).sum(axis=0)
    match_worker = first.columns[summary!=0].values
    summary_wo_zeros = summary[np.argwhere(summary).reshape(-1,1)]
    
    highest_match = ""
    #print(mat)
    #print(summary)
    #print(f"""He is looking for >>> {match_worker}""")

    if len(list(summary_wo_zeros)) != 0:
        if summary_wo_zeros.max() != summary_wo_zeros.min():
            highest_match = first.columns[np.argmax(summary)]
    return match_worker, highest_match


def find_matching_key_word(message, df):
    results = []
    for key_word in df[df.columns[0]]:
        if any([isMatch_1_word(word,key_word) for word in message]):
            results.append(key_word)
        if any([isMatch_many_words(key_word, two_word) for two_word in get_two_word(message)]):
            results.append(key_word)
    return results

def step_1(raw_message, message, bad_keywords):
    matched_words = []
    hard_coded = ['class a ', 'class b ', 'class c ']
    one_words = bad_keywords[bad_keywords['name'].apply(lambda x: not len(x.split())>1)]["name"]
    two_words = bad_keywords[bad_keywords['name'].apply(lambda x: len(x.split())>1)]["name"]
    
    for word in hard_coded: 
        if word in raw_message.lower(): matched_words.append(word)
    for word in pre_process(raw_message):
        if any([isMatch_1_word(word, word2, thresh=85) for word2 in one_words]): matched_words.append(word)
    for word in two_words:
        if any([isMatch_many_words(word, two_word) for two_word in message]): matched_words.append(word)
    
    return matched_words

In [124]:
xls = pd.ExcelFile('strings stems.xlsx')
first = pd.read_excel(xls, 'first').dropna(axis=1, how='all').dropna(axis=0, how='all')
parent_second = pd.read_excel(xls, 'parent second').dropna(axis=1, how='all').dropna(axis=0, how='all')
bad_keywords= pd.read_excel(xls, 'bad keywords').dropna(axis=1, how='all').dropna(axis=0, how='all')
at_least_another= pd.read_excel(xls, 'at least another').dropna(axis=1, how='all').dropna(axis=0, how='all')
tutoring = pd.read_excel(xls, 'Tutoring').dropna(axis=1, how='all').dropna(axis=0, how='all')
payments = pd.read_excel(xls, 'payments').dropna(axis=1, how='all').dropna(axis=0, how='all')
contract = pd.read_excel(xls, 'contract').dropna(axis=1, how='all').dropna(axis=0, how='all')

second_extract =  pd.read_csv('secondextract.csv').dropna(axis=1, how='all').dropna(axis=0, how='all')

pre_process_df_keywords(first)
pre_process_df_keywords(parent_second)
pre_process_df_keywords(bad_keywords)

print('x' in parent_second["sound effects"].values)
contract

False


,name
0,contract
1,NDA


In [125]:
raw_message = """Hello,

We're looking for someone to become our new video editor for our entrepreneur video series. (Example: https://drive.google.com/open?id=1l2yGfmvyhGcSSyDi23Xl9PNe-AEKnjcv) We currently have over 30 videos (all are between 5 and 15 minutes long) that need to be edited, and we'll be asking you to edit more and more of them the longer we work together.

You should be able to make high quality, HD video quickly and without any errors. The work is simple, but it needs your attention to make sure it feels personal, is well put together, and is aligned with our overall vision.

Make sure that you include a couple of examples of similar style videos that you have edited in the past in your proposal. Also, include your price per edited video in your proposal. Any proposals that do not include a price per video in them will be dismissed immediately.

tutor NDA cash app

Here is examples on Pinterest.
""".lower()
message = pre_process(raw_message)

print("Breaking_2") if step_1(raw_message, get_two_word(message), bad_keywords) else False
print(step_1(raw_message, get_two_word(message), bad_keywords))


find_workers(raw_message, message, parent_second), find_workers(raw_message, message, first)

[]


((array(['video editing', 'creating similar websites'], dtype=object),
  'video editing'),
 (array(['a video editor'], dtype=object), ''))

In [101]:
fuzz.partial_ratio("solid works", "solidworks")

90

In [39]:
'class a ' in 'class aside'

False

In [40]:
len("""
a v """.lower().split())
from ner import NER

In [41]:
len("""
photoshop needed
""".lower())
extractor = NER()

In [42]:
from ner import NER
text = """compensation: $60/hour
"""
def append_sapces(text):
    if text.find("-"): 
        (text[:text.find("-")] + " " + text[text.find("-")] + " " + text[text.find("-")+1:])
    if text.find("/"):
        (text[:text.find("/")] + " " + text[text.find("/")] + " " + text[text.find("/")+1:])
    return text

In [94]:
extractor.extract_entities_dict(append_sapces(text))

{'money': '60'}

In [95]:
extractor.extract_entities_dict(append_sapces((text)))

{'money': '60'}

In [127]:
find_matching_key_word(message, payments)

['cash app']

In [82]:
fuzz.ratio("intern","international")

63